In [1]:
import os
import pandas as pd
import patsy
import numpy as np

In [2]:
if not os.getcwd().endswith('assignment3'):
    os.chdir(os.path.join('assignment3'))
# read the file
baseball = pd.read_csv('baseball.csv')

baseball.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1232 entries, 0 to 1231
Data columns (total 15 columns):
Team            1232 non-null object
League          1232 non-null object
Year            1232 non-null int64
RS              1232 non-null int64
RA              1232 non-null int64
W               1232 non-null int64
OBP             1232 non-null float64
SLG             1232 non-null float64
BA              1232 non-null float64
Playoffs        1232 non-null int64
RankSeason      244 non-null float64
RankPlayoffs    244 non-null float64
G               1232 non-null int64
OOBP            420 non-null float64
OSLG            420 non-null float64
dtypes: float64(7), int64(6), object(2)
memory usage: 144.5+ KB


### Problem 1.1
Q. How many team/year pairs are there in the whole dataset?

In [3]:
len(baseball.index)

1232

### Problem 1.2 
Q. What is the total number of years included in this dataset.

In [4]:
len(baseball['Year'].unique())

47

### Problem 1.3
Replace the 'baseball' dataframe with a data frame limited to teams that made the playoffs. 

Q. How many team/year pairs are included in the new dataset?

In [5]:
baseball = baseball.query('Playoffs == 1')

In [6]:
len(baseball.index)

244

### Problem 1.4
Q. What is the possible number of teams making it to the playoffs? this number changed over the years.

In [7]:
baseball.groupby('Year')['Team'].count().unique()

array([ 2,  4,  8, 10], dtype=int64)

### Problem 2.1 - 2.4
Add the predictor variable NumCompetitors to the baseball data frame.

In [8]:
year_nteams = baseball.groupby('Year')['Team'].count()

In [9]:
for year in year_nteams.index:
    baseball.loc[baseball['Year'] == year, 'NumCompetitors'] = year_nteams[year]

Q. How many playoff team/year pairs are there in our dataset from years where 8 teams were invited to the playoffs?

In [10]:
(baseball['NumCompetitors'] == 8).sum()

128

### Problem 3.1
Add a variable named WorldSeries to the baseball dataFrame. WorldSeries takes value 1 if a team won the World Series in the indicated year and a 0 otherwise. 

In [11]:
baseball['WorldSeries'] = (baseball['RankPlayoffs'] == 1).astype(np.int)

Q. How many observations do we have in our dataset where a team did NOT win the World Series?

In [12]:
(baseball['WorldSeries'] == 0).sum()

197

### Problem 3.2
Q. Which of the variables is a significant predictor (P < 0.05) of the WorldSeries variable in a bivariate logistic regression model? Note that you have to build 12 models to answer this question!

In [13]:
bvreg_vars = baseball.columns.drop(['WorldSeries', 
                                    'Team', 'Playoffs', 
                                    'G', 'RankPlayoffs'])
bvreg_vars

Index(['League', 'Year', 'RS', 'RA', 'W', 'OBP', 'SLG', 'BA', 'RankSeason',
       'OOBP', 'OSLG', 'NumCompetitors'],
      dtype='object')

In [17]:
from statsmodels.discrete.discrete_model import Logit
import patsy

y = baseball['WorldSeries']

In [65]:
for variable in bvreg_vars:
    print('var: ', variable)
    try:
        X = patsy.dmatrix(str(variable), data=baseball)
        mod = Logit(y, X).fit()
        x1_coeff = mod.summary2().tables[1]['P>|z|'][1]
        print('{}: P = {:0.5f}, AIC = {:0.2f}'.format(variable, x1_coeff, mod.aic))
        
    except:
        print('Fit failed')

var:  League
Optimization terminated successfully.
         Current function value: 0.489518
         Iterations 5
League: P = 0.62645, AIC = 242.88
var:  Year
Optimization terminated successfully.
         Current function value: 0.467932
         Iterations 6
Year: P = 0.00115, AIC = 232.35
var:  RS
Optimization terminated successfully.
         Current function value: 0.486587
         Iterations 6
RS: P = 0.20134, AIC = 241.45
var:  RA
Optimization terminated successfully.
         Current function value: 0.479270
         Iterations 6
RA: P = 0.02621, AIC = 237.88
var:  W
Optimization terminated successfully.
         Current function value: 0.482603
         Iterations 6
W: P = 0.05773, AIC = 239.51
var:  OBP
Optimization terminated successfully.
         Current function value: 0.487748
         Iterations 6
OBP: P = 0.29592, AIC = 242.02
var:  SLG
Optimization terminated successfully.
         Current function value: 0.482025
         Iterations 6
SLG: P = 0.05043, AIC = 239.23

### Problem 4.1
Build a model using all of the variables that you found to be significant in the bivariate models. 

Q. How many variables are significant in the combined model?

In [66]:
X = patsy.dmatrix('Year + RA + RankSeason + NumCompetitors', data=baseball)
mod = Logit(y, X).fit()
mod.summary2()

Optimization terminated successfully.
         Current function value: 0.463874
         Iterations 6


<table class="simpletable">
<tr>
        <td>Model:</td>              <td>Logit</td>      <td>Pseudo R-squared:</td>   <td>0.053</td> 
</tr>
<tr>
  <td>Dependent Variable:</td>    <td>WorldSeries</td>         <td>AIC:</td>        <td>236.3707</td>
</tr>
<tr>
         <td>Date:</td>        <td>2017-03-28 00:23</td>       <td>BIC:</td>        <td>253.8565</td>
</tr>
<tr>
   <td>No. Observations:</td>         <td>244</td>        <td>Log-Likelihood:</td>   <td>-113.19</td>
</tr>
<tr>
       <td>Df Model:</td>              <td>4</td>            <td>LL-Null:</td>       <td>-119.56</td>
</tr>
<tr>
     <td>Df Residuals:</td>           <td>239</td>         <td>LLR p-value:</td>    <td>0.012557</td>
</tr>
<tr>
      <td>Converged:</td>           <td>1.0000</td>           <td>Scale:</td>        <td>1.0000</td> 
</tr>
<tr>
    <td>No. Iterations:</td>        <td>6.0000</td>              <td></td>              <td></td>    
</tr>
</table>
<table class="simpletable">
<tr>
    <td></td>     <th>Coef.</th>  <th>Std.Err.</th>    <th>z</th>     <th>P>|z|</th>  <th>[0.025</th>   <th>0.975]</th> 
</tr>
<tr>
  <th>const</th> <td>12.5874</td>  <td>53.6477</td> <td>0.2346</td>  <td>0.8145</td> <td>-92.5601</td> <td>117.7349</td>
</tr>
<tr>
  <th>x1</th>    <td>-0.0061</td>  <td>0.0275</td>  <td>-0.2236</td> <td>0.8230</td>  <td>-0.0600</td>  <td>0.0477</td> 
</tr>
<tr>
  <th>x2</th>    <td>-0.0008</td>  <td>0.0027</td>  <td>-0.3007</td> <td>0.7636</td>  <td>-0.0062</td>  <td>0.0045</td> 
</tr>
<tr>
  <th>x3</th>    <td>-0.0685</td>  <td>0.1203</td>  <td>-0.5692</td> <td>0.5692</td>  <td>-0.3044</td>  <td>0.1674</td> 
</tr>
<tr>
  <th>x4</th>    <td>-0.1794</td>  <td>0.1816</td>  <td>-0.9881</td> <td>0.3231</td>  <td>-0.5353</td>  <td>0.1765</td> 
</tr>
</table>

A. None!

### Problem 4.2
Q. Which variable pairs have a high degree of correlation (a correlation greater than 0.8 or less than -0.8)? 

In [63]:
baseball[['Year', 'RA', 'RankSeason', 'NumCompetitors']].corr()

,Year,RA,RankSeason,NumCompetitors
Year,1.000000,0.476242,0.385219,0.913955
RA,0.476242,1.000000,0.399141,0.513677
RankSeason,0.385219,0.399141,1.000000,0.424739
NumCompetitors,0.913955,0.513677,0.424739,1.000000


### Problem 4.3
Build all six of the two variable models listed in the previous problem. Which model has the best AIC value (the smallest)?

In [72]:
bvreg_var_pairs = ['Year + RA',
                   'Year + RankSeason',
                   'Year + NumCompetitors',
                   'RA + RankSeason',
                   'RA + NumCompetitors',
                   'RankSeason + NumCompetitors']


for variable in bvreg_var_pairs:
    print('var: ', variable)
    try:
        X = patsy.dmatrix(variable, data=baseball)
        mod = Logit(y, X).fit()
        x1_coeff = mod.summary2().tables[1]['P>|z|'][1]
        print('{}: P = {:0.5f}, AIC = {:0.2f}'.format(variable, x1_coeff, mod.aic))

    except:
        print('Fit failed')

var:  Year + RA
Optimization terminated successfully.
         Current function value: 0.466960
         Iterations 6
Year + RA: P = 0.01603, AIC = 233.88
var:  Year + RankSeason
Optimization terminated successfully.
         Current function value: 0.466296
         Iterations 6
Year + RankSeason: P = 0.00822, AIC = 233.55
var:  Year + NumCompetitors
Optimization terminated successfully.
         Current function value: 0.464953
         Iterations 6
Year + NumCompetitors: P = 0.80344, AIC = 232.90
var:  RA + RankSeason
Optimization terminated successfully.
         Current function value: 0.475857
         Iterations 6
RA + RankSeason: P = 0.11810, AIC = 238.22
var:  RA + NumCompetitors
Optimization terminated successfully.
         Current function value: 0.464637
         Iterations 6
RA + NumCompetitors: P = 0.64313, AIC = 232.74
var:  RankSeason + NumCompetitors
Optimization terminated successfully.
         Current function value: 0.464188
         Iterations 6
RankSeason + NumC

A. NumCompetitors